# Contents
Demo of reports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join
import logging
import json

import pandas as pd
from gridemissions.load import BaData
from gridemissions.eia_api import BAs, KEYS, SRC
import gridemissions
import re
import numpy as np

In [ ]:
from gridemissions.viz import set_plots, reports

COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()

In [ ]:
file_name = join(gridemissions.config["APEN_PATH"], "data", "EBA_%s.csv")
co2 = BaData(fileNm=file_name % "co2", variable="CO2")
elec = BaData(fileNm=file_name % "elec", variable="E")

In [ ]:
file_name = "../EBA_%s.csv"
co2_vm = BaData(fileNm=file_name % "co2", variable="CO2")
elec_vm = BaData(fileNm=file_name % "elec", variable="E")

In [ ]:
ba = "CISO"
reports._plot_carbon_intensity(
    ba, co2, elec, add_title="Carbon intensity", unit="kg/MWh"
)
reports._plot_carbon_intensity(
    ba, co2_vm, elec_vm, add_title="Carbon intensity VM", unit="kg/MWh"
)

In [ ]:
end = "2021-01-01"
start = "2020-01-01"
tz_offset = "6"
co2i = pd.DataFrame(
    {
        ba: (
            co2.df.loc[start:end, co2.get_cols(ba, field="D")].values.flatten()
            / elec.df.loc[start:end, elec.get_cols(ba, field="D")].values.flatten()
        )
        for ba in co2.regions
    },
    index=co2.df.loc[start:end].index,
)
co2i.index -= pd.Timedelta(f"{tz_offset}h")

f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5 * ROW_HEIGHT))
reports.heatmap(
    co2i[ba],
    fax=(f, ax),
    cmap="RdYlGn_r",
    cbar_label=f"{ba} - kg/MWh",
    transpose=True,
)
add_watermark(ax)
ax.set_title(f"{ba}: Consumption-based carbon intensity", fontsize="large")
f.tight_layout()

In [ ]:
ba

In [ ]:
co2_ = co2_vm
elec_ = elec_vm
co2iD = (
    co2_.df.loc[:, co2_.get_cols(r=ba, field="D")].values.flatten()
    / elec_.df.loc[:, elec_.get_cols(r=ba, field="D")].values.flatten()
)
co2iG = (
    co2_.df.loc[:, co2_.get_cols(r=ba, field="NG")].values.flatten()
    / elec_.df.loc[:, elec_.get_cols(r=ba, field="NG")].values.flatten()
)
co2iD[co2iD > 2000] = np.nan
co2iG[co2iG > 2000] = np.nan

impC, expC = separate_imp_exp(co2_, ba)
impE, expE = separate_imp_exp(elec_, ba)

co2i_imp = impC / impE
co2i_exp = expC / expE

f, ax = plt.subplots(1, 1, figsize=(PAGE_WIDTH, ROW_HEIGHT))
myplot(ax, pd.Series(co2iD, index=co2_.df.index), label="Demand", color=COLORS[0])
myplot(ax, pd.Series(co2iG, index=co2_.df.index), label="Generation", color=COLORS[1])
myplot(ax, pd.Series(co2i_imp, index=co2_.df.index), label="Imports", color=COLORS[2])

In [ ]:
impE

In [ ]:
impE, expE = separate_imp_exp(elec, ba)
impE

In [ ]:
co2.get_trade_partners(ba)

In [ ]:
# Consolidate heatmap report
# Cleanup plots in this report to show usage